In [1]:
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

DATA_PATH = "D:/data/311_Service_Requests_2015.csv" # Make this the /path/to/the/data


In [2]:
col = ['Created Date', 'Incident Zip', 'Complaint Type', 'Agency', 'Agency Name']
my_type = {'Created Date': str, 'Incident Zip': float64, 'Complaint Type' : str}

In [3]:
# data_2015 = pd.read_csv(DATA_PATH, usecols=col, dtype=my_type)
data_2015 = pd.read_csv(DATA_PATH, usecols=col)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_2015.head()

,Created Date,Agency,Agency Name,Complaint Type,Incident Zip
0,03/26/2015 06:50:06 PM,DSNY,Department of Sanitation,Graffiti,11104
1,03/28/2015 04:30:52 PM,DOB,Department of Buildings,Building/Use,11361
2,04/01/2015 04:48:00 PM,DOT,Department of Transportation,Street Light Condition,NaN
3,05/04/2015 04:46:00 PM,DOT,Department of Transportation,Street Light Condition,11234
4,04/17/2015 01:50:07 PM,DSNY,Department of Sanitation,Graffiti,11236


In [5]:
data_2015.dtypes

Created Date      object
Agency            object
Agency Name       object
Complaint Type    object
Incident Zip      object
dtype: object

In [6]:
sample_col = ['Created Date', 'Incident Zip', 'Complaint Type']
data_clean = data_2015[sample_col]

In [7]:
data_clean.head()

,Created Date,Incident Zip,Complaint Type
0,03/26/2015 06:50:06 PM,11104,Graffiti
1,03/28/2015 04:30:52 PM,11361,Building/Use
2,04/01/2015 04:48:00 PM,NaN,Street Light Condition
3,05/04/2015 04:46:00 PM,11234,Street Light Condition
4,04/17/2015 01:50:07 PM,11236,Graffiti


In [8]:
len(data_clean)

2282098

In [9]:
data_clean['Incident Zip'] = data_clean['Incident Zip'].convert_objects(convert_numeric=True)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
data_clean.dtypes

Created Date       object
Incident Zip      float64
Complaint Type     object
dtype: object

In [11]:
data_clean = data_clean.dropna(axis=0) #drop NaN

In [12]:
len(data_clean)

2114618

In [13]:
complaint_type = data_clean['Complaint Type']

In [14]:
complaint_type

0                               Graffiti
1                           Building/Use
3                 Street Light Condition
4                               Graffiti
5                 Street Light Condition
7                               Graffiti
8                           Building/Use
9                               Graffiti
10                          Building/Use
11                              Graffiti
12                          Building/Use
13         General Construction/Plumbing
14                              Graffiti
15                          Building/Use
18         General Construction/Plumbing
19                          Building/Use
20                              Graffiti
21               Overgrown Tree/Branches
22                          Building/Use
23               Overgrown Tree/Branches
24               Overgrown Tree/Branches
25               Overgrown Tree/Branches
26                          Building/Use
27                              Graffiti
28              

In [15]:
# complaint_type = complaint_type.drop_duplicates(subset='Complaint Type', keep='last')
# complaint_type = complaint_type.groupby('Complaint Type')
complaint_type = complaint_type.drop_duplicates()

In [16]:
complaint_type

0                                         Graffiti
1                                     Building/Use
3                           Street Light Condition
13                   General Construction/Plumbing
21                         Overgrown Tree/Branches
38                   Root/Sewer/Sidewalk Condition
69                                New Tree Request
81                                  Taxi Complaint
85                           Street Sign - Damaged
96                                    Damaged Tree
100                            Special Enforcement
103                       Traffic Signal Condition
113                          Street Sign - Missing
119                                   Water System
122                                       Plumbing
129                               Street Condition
131                                       Elevator
140        Special Projects Inspection Team (SPIT)
216                                          Noise
221                            

In [17]:
len(complaint_type)

200

In [18]:
complaint_array = complaint_type.values

In [19]:

complaint_dic = {}
count = 0
for i in complaint_array:
    complaint_dic[i] = count
    count+=1
complaint_dic

{'AGENCY': 188,
 'APPLIANCE': 100,
 'Adopt-A-Basket': 154,
 'Advocate - Other': 194,
 'Advocate - RPIE': 199,
 'Advocate-Commercial Exemptions': 192,
 'Advocate-Personal Exemptions': 190,
 'Advocate-Prop Class Incorrect': 191,
 'Advocate-SCRIE/DRIE': 189,
 'Air Quality': 22,
 "Alzheimer's Care": 78,
 'Animal Abuse': 124,
 'Animal Facility - No Permit': 152,
 'Animal in a Park': 50,
 'Asbestos': 40,
 'BEST/Site Safety': 44,
 'Beach/Pool/Sauna Complaint': 147,
 'Bereavement Support Group': 166,
 'Bike Rack Condition': 85,
 'Bike/Roller/Skate Chronic': 138,
 'Blocked Driveway': 74,
 'Boilers': 90,
 'Bottled Water': 173,
 'Bridge Condition': 150,
 'Broken Muni Meter': 87,
 'Broken Parking Meter': 134,
 'Building Condition': 183,
 'Building/Use': 1,
 'Bus Stop Shelter Placement': 132,
 'Calorie Labeling': 170,
 'Case Management Agency Complaint': 159,
 'City Vehicle Placard Complaint': 86,
 'Collection Truck Noise': 137,
 'Construction': 73,
 'Consumer Complaint': 84,
 'Cranes and Derricks'

In [20]:
data_feature = data_clean[['Created Date', 'Incident Zip']]
data_target = data_clean[['Complaint Type']]

In [21]:
data_feature.head()
data_feature.dtypes

Created Date     object
Incident Zip    float64
dtype: object

In [22]:
data_feature.values

array([['03/26/2015 06:50:06 PM', 11104.0],
       ['03/28/2015 04:30:52 PM', 11361.0],
       ['05/04/2015 04:46:00 PM', 11234.0],
       ..., 
       ['09/02/2015 12:58:00 PM', 11001.0],
       ['09/02/2015 08:59:00 AM', 11691.0],
       ['07/16/2015 09:42:04 AM', 11222.0]], dtype=object)

In [23]:
data_feature_zip = data_clean[['Incident Zip']]

In [24]:
data_feature_zip.values

array([[ 11104.],
       [ 11361.],
       [ 11234.],
       ..., 
       [ 11001.],
       [ 11691.],
       [ 11222.]])

In [25]:
data_feature_zip.shape

(2114618, 1)

In [26]:
data_target.head()
data_target.dtypes

Complaint Type    object
dtype: object

In [27]:
data_target_array = data_target.values
data_target_array

array([['Graffiti'],
       ['Building/Use'],
       ['Street Light Condition'],
       ..., 
       ['Sewer'],
       ['Sewer'],
       ['General Construction/Plumbing']], dtype=object)

In [28]:
data_target_num = data_target.replace(complaint_dic)

In [29]:
data_target_num.head()

,Complaint Type
0,0
1,1
3,2
4,0
5,2


In [30]:
data_target_num.dtypes

Complaint Type    int64
dtype: object

In [76]:
# pd.to_numeric(data_sample_zip['Incident Zip'])
# mydf['CigarNum'] = mydf['CigarNum'].convert_objects(convert_numeric=True)
# data_sample_zip['Incident Zip'] = data_sample_zip['Incident Zip'].convert_objects(convert_numeric=True)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [33]:
data_feature_zip.shape

(2114618, 1)

In [34]:
data_target_num.shape

(2114618, 1)

- K-nearest neighbors (KNN) classification

In [35]:
from sklearn.neighbors import KNeighborsClassifier


knn_1 = KNeighborsClassifier(n_neighbors=1)

In [40]:
data_feature_zip.as_matrix()

array([[ 11104.],
       [ 11361.],
       [ 11234.],
       ..., 
       [ 11001.],
       [ 11691.],
       [ 11222.]])

In [41]:
data_target_num.as_matrix()

array([[ 0],
       [ 1],
       [ 2],
       ..., 
       [21],
       [21],
       [ 3]], dtype=int64)

In [37]:
knn_1.fit(data_feature_zip, data_target_num.values)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [ ]:
from sklearn.cross_validation import cross_val_score




In [63]:
from sklearn.cross_validation import KFold
kf = KFold(25, n_folds=5, shuffle=False)

In [64]:
kf

sklearn.cross_validation.KFold(n=25, n_folds=5, shuffle=False, random_state=None)

我只是试试